# Exercise: Babylonian Square Root

## Algorithm + Numerical precision

The Babylonians introduced a simple but clever iterative algorithm for computing the square root of a number:

> Repeat $t \leftarrow (t + x/t)/2$ until $t$ converges to $\sqrt{x}$.

Here a Julia implementation that performs the iterative procedure `N` times.

In [1]:
function sqrt_babylonian(x, N = 10)
    t = (1+x)/2
    for i = 2:N
        t = (t + x/t)/2
    end
    t
end

sqrt_babylonian (generic function with 2 methods)

**Tasks**

1. Confirm that the iterative algorithm indeed converges to the square root by comparing the output of `sqrt_babylonian` to `sqrt(big"2.0")` for `N` increasing from `1` to `10`. (Here, `big"2.0"` creates a `BigFloat` which has arbitrarily high precision.)
    - How big is the difference to the "exact" result?
    - Does increasing `N` any further help?
2. Do the same as in task 1 but vary the data type of the input number. Specifically, use `Float16(2.0)`, `Float32(2.0)`, `Float64(2.0)`, and `big"2.0"` (`BigFloat`).

In [3]:
# 1.
for i in 1:10
    println(sqrt_babylonian(2.0, i) - sqrt(big"2.0"))
end

0.08578643762690495119831127579030192143032812462305192682332026200926752153789802
0.00245310429357146983524132576942986494610571251367692682332026200926752153789802
2.123901414615797774852241245868268707030873051926823320262009267521537898020485e-06
1.594821025504709383794876399734252552739426823320262009267521537898020485288439e-12
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16
-1.253716717905021777128546450199081980731766797379907324784621019795147115605803e-16


In [4]:
# 2.
for dtype in (Float16, Float32, Float64, BigFloat)
    println("dtype = $dtype")
    for i in 1:10
        println(sqrt_babylonian(dtype(2.0), i) - sqrt(big"2.0"))
    end
    println()
end

dtype = Float16
0.08578643762690495119831127579030192143032812462305192682332026200926752153789802
0.00180206262690495119831127579030192143032812462305192682332026200926752153789802
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795
-0.0001510623730950488016887242096980785696718753769480731766797379907324784621019795

dtype = Float32
0.08578643762690495119831127579030192143032812462305192682332026200926752153789802
0.0024531837664313183858112757

The fact that our generic implementation of the babylonian square root algorithm can be executed with different numerical precisions is only a specific case of a more general concept: The combination of generic code and special data types can lead to "emergent" features. Below, we'll consider three simple but hopefully somewhat exciting examples that will make our `sqrt_babylonian`
* compute not only the square root itself but also its derivative,
* produce an analytical expression that approximates the square root, and
* propagate uncertainty in the input (according to linear error propagation theory) to the output.

And all of this **without modifying our `sqrt_babylonian` implementation above at all**.

## Automatic differentiation
A powerful number type invented by Clifford in 1873 is the [*dual number*](https://en.wikipedia.org/wiki/Dual_number). One application of these numbers is what is known as forward-mode automatic differentiation (AD) these days. Here a simple Julia implementation:

In [5]:
# D for "dual number", invented by Clifford in 1873.
struct D <: Number
    x::Float64 # value
    ϵ::Float64 # derivative
end

Base.:+(a::D, b::D) = D(a.x + b.x, a.ϵ + b.ϵ) # sum rule
Base.:-(a::D, b::D) = D(a.x - b.x, a.ϵ - b.ϵ) # sum rule
Base.:*(a::D, b::D) = D(a.x * b.x, a.x * b.ϵ + a.ϵ * b.x) # product rule
Base.:/(a::D, b::D) = D(a.x / b.x, (b.x * a.ϵ - a.x * b.ϵ)/b.x^2) # quotient rule

# Conversion and promotion of regular numbers to dual numbers
Base.convert(::Type{D}, x::Real) = D(x, zero(x))
Base.promote_rule(::Type{D}, ::Type{<:Number}) = D

The derivative of a generic Julia function (that fundamentally relies on `+`, `-`, `*`, and `/`) can know be obtained with the following function

In [6]:
# utility function for our small forward-mode AD
derivative(f::Function, x::Number) = f(D(x, one(x))).ϵ

derivative (generic function with 1 method)

**Tasks**

3. What is the analytical derivative of $\sqrt{x}$ (try to remember your calculus class 😉)?

4. Check that `derivative(sqrt_babylonian, some_number)` indeed automagically gives the correct value of the derivative.
    - Feel free to try other functions/algorithms as well! Maybe something recursive like `pow(x, n) = n <= 0 ? 1 : x*pow(x, n-1)`?

**Answer 3.:**

Derivative of $\sqrt{x}$ is $\dfrac{1}{2\sqrt{x}}$

In [7]:
# 4.
derivative(sqrt_babylonian, 2.0)

0.35355339059327373

In [8]:
1/(2*sqrt(2)) ≈ derivative(sqrt_babylonian, 2.0)

true

In [9]:
derivative(x->2*x^2, 1.23) == 4*1.23

true

In [10]:
pow(x, n) = n <= 0 ? 1 : x*pow(x, n-1)

pow (generic function with 1 method)

In [11]:
derivative(x -> pow(x,3), 2)

12.0

## Symbolics

Now, imagine your supervisor coming to your office and telling you that he thinks that the Babylonian approximation for `N=3` is equivalent to the analytical form
$$ \text{sqrt_babylonian}(x, 3) \approx \begin{equation}
\frac{7 x + 17 x^{2} + 7 x^{3}}{8 \left( 1 + x \right) \left( \frac{1}{4} + \frac{3}{2} x + \frac{1}{4} x^{2} \right)}
\end{equation} $$

Of course, he asks you to quickly confirm this.

**Tasks**

5. Is your supervisor correct?
    - Hint: Use the `Symbolics` package, in particular `@variables x` and `simplify`.
    - Note: Make sure you have `N=3` since for larger `N` problems might start to occur.

In [12]:
# 5.
using Symbolics
@variables x

for i in 1:3
    display(sqrt_babylonian(x, i))
end

(1//2)*(1 + x)

(1//2)*((1//2)*(1 + x) + x / ((1//2)*(1 + x)))

(1//2)*((1//2)*((1//2)*(1 + x) + x / ((1//2)*(1 + x))) + x / ((1//2)*((1//2)*(1 + x) + x / ((1//2)*(1 + x)))))

In [13]:
simplify(sqrt_babylonian(x, 3))

(7x + 17(x^2) + 7(x^3)) / (8(1 + x)*((1//4) + (3//2)*x + (1//4)*(x^2)))

## Uncertainty propagation

In the experimental sciences, numerical values (e.g., from measurements) are often subject to uncertainties due to systematic precision errors of the measurement devices. The Julia package [Measurements.jl](https://github.com/JuliaPhysics/Measurements.jl) provides a number type and corresponding arithmetical operations that address this situation. Specifically, the package implements linear error propagation theory, which states that given a function $f(x)$ and an input value $x_0$ with uncertainty $\Delta x_0$, the uncertainty of $f(x_0)$ is given by
$$
\Delta (f(x_0)) = \dfrac{df}{dx}(x_0) \cdot \Delta x_0,
$$
that is, the derivative of $f$ evaluated at $x_0$ multiplied by the input uncertainty $\Delta x_0$.

In [14]:
# Example:
using Measurements

x = 1.2 ± 0.1 # you can get the ± by typing \pm and then pressing the "TAB" key.
y = 3.4 ± 0.2

x + y

4.6 ± 0.22

**Tasks**

6. Try to run our sqrt algorithm with a `Measurement` as input, i.e. `sqrt_babylonian(2.0 ± 0.1)`.
    - Does it work?
    - What uncertainy do you get for the result?
7. It never hurts to check for correctness: Does the obtained uncertainty match the formula above for $f=\textrm{sqrt}$?

In [15]:
# 6..
sqrt_babylonian(2.0 ± 0.1)

1.414 ± 0.035

In [16]:
# 7.
1/(2*sqrt(2.0)) * 0.1 ≈ Measurements.uncertainty(sqrt_babylonian(2.0 ± 0.1))

true